# 台灣教育部國語辭典簡編本 網絡爬蟲
# Taiwan Ministry of Education Concise Chinese Dictionary Web Crawler/Scraper
## (for Kindle dictionary)

This notebook is scrapes the Ministry of Education (moe) website for the simplified/concise version of the chinese dictionary. [http://dict.concised.moe.edu.tw/](http://dict.concised.moe.edu.tw/) This scraper attempts to gather data into a .json file formatted similar to [this](https://github.com/g0v/moedict-data) popular repo. Though, the motivation is to gather data for a proper Traditional Chinese Kindle dictionary, which does not require as clean of data processing after the scraping so the format is not exactly the same as the mentioned repo's. 

The approach is to use the listings by stroke count to gather links to each entry and then gather at least the: title, bopomofo, pinyin, and definition. Some entries have synonyms and antonyms that are also extracted.

This notebook does require some babysitting as explained in the next markdown cell.

In [1]:
import re
from requests_html import HTMLSession
import json
import time
import random
import cellbell

C:\Users\eyeh2\anaconda3\envs\moe-dict-concised-scraper\lib\site-packages\pyglet\media\codecs\wmf.py:838: UserWarning: [WinError -2147417850] Cannot change thread mode after it is set
  warnings.warn(str(err))


It seems that the moe server generates a unique 6 character 'ccd' identifier that only works for some number of requests. (idk what this is called) See stroke_main_url below where ccd=KD6FAc. I'm not sure how to get a new ccd after the current one expires, but once it does (which the program will let you know) we must manually load up [http://dict.concised.moe.edu.tw/](http://dict.concised.moe.edu.tw/), go to an entry, and copy paste the new ccd from the url into the ccd variable below and rerun the notebook.

The below cell doesn't always return success even with the retries. After an unsuccessful run with a new ccd, let it cool down for a few seconds and keep trying.

In [196]:
session = HTMLSession()

def process_url(url, ccd):
    return re.sub(r'(?<=ccd=).{6}', ccd, url)

ccd = 'mZ2330'
# this is the main webpage that has all the buttons for number of strokes
# from clicking '筆畫索引'
stroke_main_url = 'http://dict.concised.moe.edu.tw/cgi-bin/jbdic/gsweb.cgi?ccd=KD6FAc&o=e0&&sec=sec1&brwtyp=sco&brwsimpfmt=11&field_1=sco1&field_1_value=XXX&field_2=sco2&brwsortby=wqyx&active=bhbrw'
stroke_main_url = process_url(stroke_main_url, ccd)

tries = 0
MAX_TRIES = 1
while tries < MAX_TRIES:
    print('Attempt {}/{}'.format(tries+1, MAX_TRIES))
    stroke_main = session.get(stroke_main_url)
    if len(stroke_main.html.absolute_links) > 0:
        # get all 1畫, 2畫, ... links
        stroke_urls_elems = stroke_main.html.find('[class=sco_bst_lv1]')
        stroke_urls = [url.absolute_links.pop() for url in stroke_urls_elems]
        print('Success. {} stroke urls found. (Should be exactly 33)'.format(len(stroke_urls)))
        break
    tries += 1
    time.sleep(2*random.random())
    if tries == MAX_TRIES:
        print('Attempts unsuccessful. Try using new ccd session id.')
        

Attempt 1/1
Success. 33 stroke urls found. (Should be exactly 33)


## Classes to represent a dictionary entry

The overall hierarchy is:
DictEntry:
    title: str
    radical: str
    non_radical_stroke_count: str
    stroke_count: str
    heteronyms: [
        Heteronym:
            bopomofo: str
            pinyin: str
            synonyms: str
            antonyms: str
            definitions: [
                Definition:
                    defn: str
                    quote: str
            ]
        ]

In [102]:
class Definition:
    """Lowest level class for a dictionary entry. This contains the actual
    text definition for an entry. There are supposed to be more fields in this
    class (quote, synonyms, antonyms) but were taken out for ease of
    implementation for the Kindle. The full breakdown of the definition is TBD.
    For now, this class holds all definitions and examples for an entry 
    (everything in the 釋義 table cell). Synonyms and antonyms belong to the
    Heteronym class since the extracted text has numbers labelling which
    definition it corresponds to.
    """
    defn: str

    def __init__(self, defn: str = None):
        """Initializes definition.
        """
        self.defn = defn

    def __repr__(self) -> str:
        return json.dumps(self.__dict__, default=lambda o: o.__dict__, ensure_ascii=False)

class Heteronym:
    """Represents all data for a pronunciation of a specific dictionary entry.
    For example for a DictEntry title of 不好, there would be two heteronyms 
    ㄅㄨˊ　ㄏㄠˋ and ㄅㄨˋ　ㄏㄠˇ, each with its own set of definitions/quotes/
    synonyms/antonyms.
    """
    bopomofo: str
    pinyin: str
    definitions: list
    synonyms: str
    antonyms: str

    def __init__(self):
        """Initializes empty heteronym.
        """
        self.bopomofo = None
        self.pinyin = None
        self.definitions = []
        self.synonyms = None
        self.antonyms = None

    def __repr__(self) -> str:
        return json.dumps(self.__dict__, default=lambda o: o.__dict__, ensure_ascii=False)

class DictEntry:
    """A class to represent a chinese entry in a dictionary. Each entry may
    contain different heternoyms. This class was created with easy __dict__ to 
    json conversion in a post-processing step in mind.
    
    title (str): the dictionary entry title (dictionary key)
    heteronyms (list): list of different Heteronym objects
    radical (str): the radical of the character (only for single characters)
    non_radical_stroke_count (int): (only for single characters)
    stroke_count (int): (only for single characters)
    """
    title: str
    heteronyms: list = []
    radical: str
    non_radical_stroke_count: int
    stroke_count: int
    
    def __init__(self):
        """Initializes an empty dictionary entry.
        """
        self.title = None
        self.heteronyms = []
        self.radical = None
        self.non_radical_stroke_count = None
        self.stroke_count = None

    def __eq__(self, other) -> bool:
        """Primarily used to lookup and append heteronyms to existing entries.
        """
        if isinstance(other, DictEntry):
            return self.title == other.title
        return False
        
    def __repr__(self) -> str:
        # return self as __dict__, and return sub-objects as __dict__'s too
        return json.dumps(self.__dict__, default=lambda o: o.__dict__, ensure_ascii=False)


## Helper Functions

In [103]:
# html snippet of the 'example' image used to dilineate definition and examples/quotes
example_snippet = '<img src="/jbdic/cfont/9b51._104_0.gif" border="0" alt="&#x4F8B;" class="fontimg" id="@extrafont" />'
# html snippet used to express 一 in 注音
i_snippet = '<font class="zinspc1">ㄧ</font>'

def parse_def_page(def_page, existing_entry: DictEntry = None) -> DictEntry:
    """Given a requests_html page of a moe dictionary entry, parse the data and
    grab information like title, radical, stroke counts, synonyms, antonyms,
    and definitions. An existing DictEntry may be given to append a heteronym
    in place.
    Inputs:
        def_page (requests_html page): page of a moe dictionary entry
        existing_entry (DictEntry): An existing entry for a heteronym to be 
            appended to
    """
    # grab headers on the left column
    row_headers = def_page.html.find('[class=std1]')
    # grab data on the right column
    row_data = def_page.html.find('[class=std2]')

    entry = DictEntry()
    if existing_entry:
        entry = existing_entry

    # this page represents a heteronym, so always create a new one
    heter = Heteronym()

    # process each row to build up heteronym/definition
    for i in range(len(row_headers)):
        header = row_headers[i]
        data = row_data[i]

        # process each row by looking for keywords in the header 
        
        # look for first row by seraching for a pronunciation sound file element
        if len(header.find('[id=layoutwav]')) > 0:
            # get rid of enclosing brackets
            entry.title = data.text[1:data.text.find('】')]

            # if this is a single character, then also get radical and stroke counts
            # e.g. 【不】一-3-4
            if '-' in data.text:
                entry.radical = data.text[data.text.find('】')+1]
                dash_1 = data.text.find('-') + 1
                dash_2 = data.text.find('-', dash_1)
                entry.non_radical_stroke_count = int(data.text[dash_1:dash_2])
                entry.stroke_count = int(data.text[dash_2+1:])
        elif '注音' in header.text:
            # replace weird looking | for better looking 一
            data.html = data.html.replace(i_snippet, '一')
            # note: use .full_text here to avoid weird \n placed everywhere with .text
            heter.bopomofo = data.full_text
            # print(data.text)
        elif '漢語拼音' in header.text:
            # note: use .full_text here to avoid weird \n placed everywhere with .text
            heter.pinyin = data.full_text
            # print(data.text)
        elif '相似詞' in header.text:
            heter.synonyms = data.text
            # print(data.text)
        elif '相反詞' in header.text:
            heter.antonyms = data.text
            # print(data.text)
        elif '釋義' in header.text:
            # replace the example '例' image html with regular text instead
            EX_TAG = '例:'
            data.html = data.html.replace(example_snippet, EX_TAG)            

            # there is some inconsistency in the formatting of pages
            # for ordered lists, sometimes they are in a <ol><li></li></ol> list,
            # other times it is just a big <p> element with hardcoded <br> and 
            # numbers

            # default is second case (big <p> with hardcoded <br> and numbers)
            defn = data.text

            # take care of first case (<ol></ol>)
            lis = data.find('li')
            if len(lis) > 0:
                defns = [li.text for li in lis]
                # reconstruct text in same style as second case
                defn = ''
                for j in range(len(defns)):
                    # e.g. 2.第二解釋\n
                    defn += '{}.{}\n'.format(str(j+1), defns[j])            

            # add this newly constructed definition to heteronym
            heter.definitions.append(Definition(defn))

            # print(data.text)
    
    # add this newly constructed heteronym to existing list (likely empty)
    entry.heteronyms.append(heter)
    
    return entry

def del_none(d):
    """
    Delete keys with the value ``None`` in a dictionary, recursively.

    This alters the input so you may wish to ``copy`` the dict first.
    """
    # For Python 3, write `list(d.items())`; `d.items()` won’t work
    # For Python 2, write `d.items()`; `d.iteritems()` won’t work
    for key, value in list(d.items()):
        if value is None:
            del d[key]
        elif isinstance(value, dict):
            del_none(value)
    return d  # For convenience

def remove_nulls(d):
    return {k: v for k, v in d.items() if v is not None}

## Main

Depth first search to avoid session timeout issues complicating continuing runs. Goes in increasing stroke count order and saves all entries for a root character before continuing to next. Saves all entries for a stroke count in a separate file.

This is the function to keep an eye on for babysitting. Watch for assertion error saying it needs a new ccd. To fix:
1. Go visit a dictionary entry on the moe [website](http://dict.concised.moe.edu.tw/) and copy over the ccd from the url into the ccd variable in the first cell. 
2. Change the starting index in range(len(stroke_urls)) in the first line of code below to restart whatever stroke count it was last working on.
3. Rerun whole notebook with new ccd.

Edit: found that can normally resume operation without inputting new ccd if the connection is ended by remote. i.e. pattern i noticed was could rerun for a decent amount of time if it wasn't the assertion error that I raise, in which case should update the ccd.

In [199]:
# runs always start at this index
stroke_start_ind = 11
# set to true if picking up in the middle of stroke_start_ind run
pickup_previous = True
# index of the character index of this stroke_start_ind to run first
root_char_start_ind = 268


# just to keep tally of how many entries were processed each run
run_entry_count = 0

try:
    # scrape and process in increasing total stroke count
    # save entries for each stroke count into separate file
    # this processing is stroke-centric to segment and continue runs since the 
    # session seems to expire after a certain number of requests
    rng_stroke = range(stroke_start_ind, len(stroke_urls))
    for stroke_ind in rng_stroke:
        # holds all the DictEntry objects representing each entry in the dictionary
        all_entries = []
        # all_entry_titles = []

        if pickup_previous:
            f = open("entries_{}.txt".format(stroke_ind+1), "r", encoding='utf-8')
            all_entries = [e for e in json.load(f) if 'title' in e]
            # all_entry_titles = [e['title'] for e in all_entries]
        
        print('processing {} / {} 筆畫 urls'.format(stroke_ind+1, len(stroke_urls)))
        # get urls for all root characters of this # of strokes 
        # e.g. 1 stroke : 一，乙
        #      2 strokes: 丁，七 ...
        single_stroke_url = stroke_urls[stroke_ind]
        root_char_page = session.get(single_stroke_url)
        root_char_elems = root_char_page.html.find('[class=sco_bst_lv2]')

        assert len(root_char_elems) > 0, \
            'NO ROOT CHARS FOUND -- session probably expired, please update ccd in first cell'
    
        # grab the actual urls
        root_char_urls = [e.absolute_links.pop() for e in root_char_elems]
        # grab the actual root character in string form
        root_char_strs = [e.text for e in root_char_elems]
        print(len(root_char_urls), 'root characters')
        # for i in range(len(root_char_strs)):
        #     print(i, root_char_strs[i])
        # break

        rng_root_char = range(root_char_start_ind, len(root_char_urls)) if pickup_previous else range(len(root_char_urls))
        pickup_previous = False

        for root_char_ind in rng_root_char:
            # get urls for all entries starting with a certain root character
            # e.g. for 一: 一，一把罩，一把抓，一波 ...

            # start with page one (of possibly many pages of entries)
            root_char_page_url = root_char_urls[root_char_ind]
            is_pages_left = True
            current_page = 1 # used to find url of next page

            while is_pages_left:
                print('{} ({}/{} root chars for {} 畫) page {}'.format(root_char_strs[root_char_ind], root_char_ind+1, len(root_char_urls), stroke_ind+1, current_page))
                # print(root_char_page_url)
                
                entry_page = session.get(root_char_page_url)
                # get all entry links, which are 'slink' classes but without titles 
                # (which are pagination links)
                entry_elems = entry_page.html.find('[class=slink]:not([title])')

                assert len(entry_elems) > 0, \
                    'NO ENTRIES FOUND -- session probably expired, please update ccd in first cell'

                # for each entry, go into url and extract definition
                for e in entry_elems:
                    entry_url = e.absolute_links.pop()
                    print('found entry {} ({}/{})'.format(e.text, root_char_ind+1, len(root_char_urls)))
                    def_page = session.get(entry_url)

                    assert def_page is not None, \
                        'NO ENTRIES FOUND -- session probably expired, please update ccd in first cell'
                    
                    to_search = DictEntry()
                    to_search.title = e.text
                    # see if title exists already, if so, need to add as extra heteronym
                    if to_search in all_entries:
                        print('adding heteronym {}'.format(e.text))
                        # is a heteronym (different pronunciation e.g. 樂:ㄌㄜˋ / ㄩㄝˋ)
                        # find existing entry
                        existing_entry = all_entries[all_entries.index(to_search)]
                        # method should update entry in place
                        parse_def_page(def_page, existing_entry)
                    else:
                        all_entry_titles.append(e.text)
                        new_entry = parse_def_page(def_page)
                        all_entries.append(new_entry)
                        # print(new_entry)
                    # time.sleep(random.random())
                    run_entry_count += 1

                # get url for next page
                next_page_elem = entry_page.html.find('[title="{}"]'.format(current_page + 1), first=True)
                if next_page_elem is None:
                    is_pages_left = False
                    print('No more pages, moving on to next root character')
                else:
                    root_char_page_url = next_page_elem.absolute_links.pop()
                    current_page += 1

                # save entry results after each page (~10 entries)
                print('saving results...', end='')
                f = open("entries_{}.txt".format(stroke_ind+1), "w", encoding='utf-8')
                # convert to json string then load back into dict while cleaning out
                # None fields (like most synonyms/antonyms/radical/stroke counts)
                json_string = json.dumps(all_entries, default=lambda o: o.__dict__, ensure_ascii=False)
                cleaned_dic = json.loads(json_string, object_hook=remove_nulls)
                f.write(json.dumps(cleaned_dic, indent=4, ensure_ascii=False))
                f.close()
                print('done')
        #         break
        #     break
        # break
except Exception as e:
    print(e)
    cellbell.ding()
    print(run_entry_count, ' total entries processed this run.')
        

processing 12 / 33 筆畫 urls
529 root characters
猶 (269/529 root chars for 12 畫) page 1
found entry 猶 (269/529)
found entry 猶太屯墾區 (269/529)
found entry 猶可 (269/529)
found entry 猶如 (269/529)
found entry 猶在 (269/529)
found entry 猶存 (269/529)
found entry 猶疑 (269/529)
found entry 猶豫 (269/529)
No more pages, moving on to next root character
saving results...done
琛 (270/529 root chars for 12 畫) page 1
found entry 琛 (270/529)
No more pages, moving on to next root character
saving results...done
琢 (271/529 root chars for 12 畫) page 1
found entry 琢 (271/529)


KeyboardInterrupt: 

In [ ]:
# tally of entries of each run just to get an idea of timeout rates
453
945
101
601
287
127
34
42
118
490
515
278
1046
260
360
1072
717
465
1555
609
758

609

## Post processing
moved all original files to original_scraped/ and copied new version to corrected/ for whatever errors found in following steps (documented in README)

load all files first

In [40]:
import json
import os
import re

data = []

for i in range(1,34):
    path = 'corrected/entries_{}.txt'.format(i)
    if os.path.isfile(path):
        f = open(path, 'r', encoding='utf-8')
        data.extend(json.load(f))
        f.close()
        print(path, 'added')
    else:
        print(path, 'not found')

corrected/entries_1.txt added
corrected/entries_2.txt added
corrected/entries_3.txt added
corrected/entries_4.txt added
corrected/entries_5.txt added
corrected/entries_6.txt added
corrected/entries_7.txt added
corrected/entries_8.txt added
corrected/entries_9.txt added
corrected/entries_10.txt added
corrected/entries_11.txt added
corrected/entries_12.txt added
corrected/entries_13.txt added
corrected/entries_14.txt added
corrected/entries_15.txt added
corrected/entries_16.txt added
corrected/entries_17.txt added
corrected/entries_18.txt added
corrected/entries_19.txt added
corrected/entries_20.txt added
corrected/entries_21.txt added
corrected/entries_22.txt added
corrected/entries_23.txt added
corrected/entries_24.txt added
corrected/entries_25.txt added
corrected/entries_26.txt added
corrected/entries_27.txt added
corrected/entries_28.txt added
corrected/entries_29.txt added
corrected/entries_30.txt added
corrected/entries_31.txt added
corrected/entries_32.txt added
corrected/entries

### Reorder pronunciations according to labelled 一，二，三...

In [41]:
# longest entry is 和 with 6 entries so rlly only need to go up to 6
num_lut = {'一': 0, '二': 1, '三': 2, '四': 3, '五': 4, '六': 5, '七': 6, '八': 7, '九': 8, '十':9}

hets = None

try:
    for i in range(len(data)):
        hets = data[i]['heteronyms']
        if len(hets) > 1:
            new_hets = [None] * len(hets)
            for j in range(len(hets)):
                bpmf = hets[j]['bopomofo']
                ind = num_lut[bpmf[1]]
                new_hets[ind] = hets[j]
            data[i]['heteronyms'] = new_hets
except BaseException as e:
    print(e)
    print(hets)


### Remove links to other pronunciations in each definition
for example for 傍

(一)ㄅㄤˋ

    "defn": "靠近、依附。例:依山傍水\n(二)ㄆㄤˊ\npán\ng\n\n(三)ㄅㄤ\nbān\ng"

(二)ㄆㄤˊ

    "defn": "旁邊，通「旁」。例:「賜酒大王之前，執法在傍。」（《史記．卷一二六．滑稽傳．淳于髡傳》）\n(一)ㄅㄤˋ\nbàn\ng\n\n(三)ㄅㄤ\nbān\ng"

(三)ㄅㄤ

    "defn": "接近某一時間點。例:傍午、傍晚\n(一)ㄅㄤˋ\nbàn\ng\n\n(二)ㄆㄤˊ\npán\ng"

get rid of those (一) (二) ... in the definitions


In [42]:
txt = 0
nums = list(num_lut.keys())
try:
    for i in range(len(data)):
        txt = data[i]
        hets = data[i]['heteronyms']
        # if there is more than 1 pronunciation then there are those pesky links
        if len(hets) > 1:
            # go through each pronunciation
            for j in range(len(hets)):
                # always only one definition in this implementation
                defn = hets[j]['definitions'][0]['defn']
                # print(defn)
                # search for existence of any number 1 - 10
                # for n in nums:
                    # tosearch = '(' + n + ')'
                    # ind = defn.find(tosearch)
                    # if ind != -1:
                    #     # only keep up to the found link
                    #     defn = defn[:ind]
                    #     print(defn)
                pat = r'[(][一|二|三|四|五|六|七|八|九|十][)]'
                found = [m.start(0) for m in re.finditer(pat, defn)]
                if len(found) > 0:
                    # get index of first link to other pronunciation
                    # which is the (len(hets)-1)-th to last occurence
                    # e.g. if some entry has 3 heteronyms,
                    #      the defn's will have links to the other 2 heteronyms
                    #      at the very end of the defn, so get second to last
                    # some definitions have (#) as part of their def so have to
                    # do it this way
                    ind = found[-(len(hets)-1)]
                    print(data[i]['title'])
                    defn = defn[:ind]
                    # print(data[i])
                data[i]['heteronyms'][j]['definitions'][0]['defn'] = defn

except BaseException  as e:
    print(e)
    print(txt)

一行
乜
乜
了
上
下水
亡
勺
大
大家
大汗
大汗
女
女
子
工夫
不好
不勝
不
予
予
亢
什
仇
公道
公差
公差
六
六
分子
分子
切口
切口
勾
化
匹
午
天道
天道
尺
屯
心思
手卷
手卷
文
比
比
氏
爪
爪
父
且
仔
兄弟
出處
出處
功夫
占
卡
句
叨
只
召
可
外傳
外傳
本事
正
正當
正當
生意
生意
疋
疋
石
任
吃
合
同行
同行
地
地道
地道
地下
圳
圳
好學
好學
好吃
好吃
好事
好玩
好玩
字號
字號
并
扛
扛
朴
朴
汗
汗
百
羊
色
艾
虫
虫
行
行
衣
亨
亨
伯
估
估
伺
伺
伽
伽
佛
佛
作
刨
吧
吧
吭
吭
呀
告
囤
囤
囪
囪
坊
完了
完了
弄
弟
彷
彷
忪
忪
抓
折
更
杓
肚
肚
肚子
見
角
谷
足
身
車
那
那般
那般
那裡
那些
那些
那樣
邪
乖乖
些
兒
兒
其
卒
卷
呢
呱
呱
呵
呼號
呼號
和
和
和
和
咎
奇
妻
妻
妻子
妻子
姊
姊
姑娘
姑娘
委
宓
宓
宛
底
怔
怔
所長
所長
抵
拂
拈
拈
拓
拓
拗
拗
放
於
於
東西
枕
枝
氓
氓
沮
泄
泌
法
泡泡
泡泡
的
的
的
知
空檔
舍
芾
芾
虎
阿
青
亟
亟
便
便當
便當
便宜
俛
俛
俟
俟
冒
剌
南
咯
咯
咯
咱
咳
咳
咻
咽
咽
哀樂
哀樂
哄
哇
哇
哈
契
姥
姥
孩兒
屏
度量
度量
待
思
恫
恫
扁
括
拾
挑空
挑空
施
施
施
星星
枸
枸
枸
查
歪
洒
洒
洗
洞
洩
炮
炸
為
甚
相親
相親
相稱
相稱
省事
省事
看
祇
祇
紅
背
胖
胖
若
茄
虺
虺
計量
計量
酊
酊
重犯
重犯
降服
限量
限量
革
風頭
風
食
食
倆
個
們
們
倔
倔
倘
倘
倡
員
哦
哦
哩
哩
哩
哪
埋
夏
娜
娜
娩
娩
孫
孫子
家
射
射
差
悖
悖
捎
捎
旁
晃
柴
栖
栖
桔
桔
殷
畜
祕
秘
秘
秤
脈
般
般
茲
茲
衰
釘
釘
骨
骨
高中
高中
鬲
鬲
假
偈
副
勒
匙
匙
參
參
啊
啊
啞
啦
啦
圈
埤
埤
培
培
婁
宿
宿
將
尉
崗
強辯
強辯
強記
強記
得
得
得了
從
從
徠
徠
悉數
悉數
掃
掄
掄
排頭
排頭
排
掖
掙
掙
教
斜
斜
旋
棄養
棄養
欸
欸
殺
涼
混
率
脯
荷
荼
莊家

## Save post-processed file

In [43]:
f = open('dict_concised.json', "w", encoding='utf-8')
f.write(json.dumps(data, indent=4, ensure_ascii=False))
f.close()